In [1]:
import geopandas as gpd
import pandas as pd
from scipy.spatial import distance_matrix
import numpy as np
import geopandas as gpd

In [2]:
demand = pd.read_csv('../data/curated/train_demand/daily_trains_demand_post_covid.csv')
stations = gpd.read_file('../data/curated/train_demand/mean_daily_trains_demand_post_covid.gpkg')
stations = stations.to_crs("EPSG:7855")

In [3]:
def get_weight(df, col):
    coords = df['geometry'].apply(lambda geom: (geom.x, geom.y)).tolist()
    dist_matrix = distance_matrix(coords, coords)
    dist_df = pd.DataFrame(dist_matrix, index=df[col], columns=df[col])
    variance = np.var(dist_matrix.flatten())
    W = np.exp(-1 * np.power(dist_df,2) / (2 * variance))
    return W

In [4]:
get_weight(stations, 'Station_Name')

Station_Name,Flinders Street,Southern Cross,Richmond,Melbourne Central,Parliament,Flagstaff,Footscray,South Yarra,Caulfield,North Melbourne,...,Wattle Glen,Crib Point,Hastings,Stony Point,Bittern,Somerville,Morradoo,Leawarra,Tyabb,Baxter
Station_Name,,,,,,,,,,,,,,,,,,,,,
Flinders Street,1.000000,0.996100,0.990506,0.997899,0.998083,0.996914,0.929375,0.979417,0.833766,0.985951,...,0.257721,0.000185,0.001049,0.000128,0.000489,0.007071,0.000287,0.035627,0.003115,0.014668
Southern Cross,0.996100,1.000000,0.975114,0.996348,0.991362,0.998801,0.956182,0.962885,0.798733,0.994964,...,0.231111,0.000162,0.000928,0.000111,0.000434,0.006296,0.000254,0.032351,0.002758,0.013170
Richmond,0.990506,0.975114,1.000000,0.982758,0.990890,0.977291,0.873164,0.994639,0.889326,0.954755,...,0.286509,0.000257,0.001413,0.000180,0.000660,0.009234,0.000394,0.044303,0.004141,0.018758
Melbourne Central,0.997899,0.996348,0.982758,1.000000,0.998251,0.999116,0.942740,0.965484,0.802557,0.992515,...,0.266551,0.000141,0.000823,0.000097,0.000379,0.005753,0.000221,0.030070,0.002494,0.012123
Parliament,0.998083,0.991362,0.990890,0.998251,1.000000,0.995183,0.922188,0.974979,0.825937,0.983641,...,0.283885,0.000157,0.000910,0.000109,0.000419,0.006313,0.000245,0.032460,0.002751,0.013209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Somerville,0.007071,0.006296,0.009234,0.005753,0.006313,0.005725,0.002989,0.012509,0.033699,0.004574,...,0.000309,0.594175,0.846249,0.547917,0.724681,1.000000,0.652094,0.852701,0.968389,0.971351
Morradoo,0.000287,0.000254,0.000394,0.000221,0.000245,0.000222,0.000106,0.000587,0.002089,0.000170,...,0.000005,0.992961,0.941555,0.973376,0.990807,0.652094,1.000000,0.337429,0.796268,0.511753
Leawarra,0.035627,0.032351,0.044303,0.030070,0.032460,0.029935,0.017315,0.056713,0.124377,0.024839,...,0.002162,0.288870,0.527471,0.253050,0.404651,0.852701,0.337429,1.000000,0.716783,0.949044


In [5]:
SA2 = gpd.read_file('../data/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')
SA2 = SA2[SA2['GCC_NAME21'] == 'Greater Melbourne']
SA2['geometry'] = SA2['geometry'].to_crs('EPSG:7855')

In [6]:
SA2['centre'] = SA2.centroid
SA2['SA2_NAME21'] = SA2['SA2_NAME21'].apply(lambda x: x + ' (SA2)')
SA2 = SA2[['SA2_NAME21', 'centre']]
SA2.columns = ['name', 'geometry']

In [7]:
stations = stations[['Station_Name', 'geometry']]
stations.columns = ['name', 'geometry']

In [8]:
combined = pd.concat([SA2, stations])

In [9]:
get_weight(combined, 'name')

name,Brunswick East (SA2),Brunswick West (SA2),Pascoe Vale South (SA2),Brunswick - North (SA2),Brunswick - South (SA2),Coburg - East (SA2),Coburg - West (SA2),Alphington - Fairfield (SA2),Thornbury (SA2),Northcote - East (SA2),...,Wattle Glen,Crib Point,Hastings,Stony Point,Bittern,Somerville,Morradoo,Leawarra,Tyabb,Baxter
name,,,,,,,,,,,,,,,,,,,,,
Brunswick East (SA2),1.000000,0.984277,0.968813,0.996022,0.995888,0.989032,0.981772,0.972498,0.987635,0.989086,...,0.481937,0.000492,0.002075,0.000367,0.001086,0.010395,0.000704,0.040830,0.005212,0.019210
Brunswick West (SA2),0.984277,1.000000,0.992086,0.994685,0.995042,0.983403,0.990732,0.917843,0.950414,0.948205,...,0.406545,0.000350,0.001518,0.000256,0.000795,0.007802,0.000508,0.032139,0.003849,0.014715
Pascoe Vale South (SA2),0.968813,0.992086,1.000000,0.986843,0.979155,0.986234,0.996533,0.895254,0.940604,0.926689,...,0.425309,0.000211,0.000963,0.000154,0.000492,0.005275,0.000310,0.023092,0.002531,0.010211
Brunswick - North (SA2),0.996022,0.994685,0.986843,1.000000,0.997524,0.994649,0.993833,0.950596,0.976943,0.973171,...,0.461268,0.000372,0.001612,0.000275,0.000837,0.008314,0.000537,0.033885,0.004107,0.015609
Brunswick - South (SA2),0.995888,0.995042,0.979155,0.997524,1.000000,0.985119,0.984959,0.949070,0.970039,0.972961,...,0.433634,0.000472,0.001994,0.000350,0.001051,0.009947,0.000678,0.039403,0.004983,0.018449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Somerville,0.010395,0.007802,0.005275,0.008314,0.009947,0.006590,0.005856,0.014045,0.009746,0.013041,...,0.002081,0.671816,0.880242,0.631473,0.781879,1.000000,0.721301,0.885366,0.975755,0.978035
Morradoo,0.000704,0.000508,0.000310,0.000537,0.000678,0.000396,0.000347,0.000992,0.000631,0.000915,...,0.000089,0.994617,0.955027,0.979592,0.992968,0.721301,1.000000,0.436002,0.840234,0.599371
Leawarra,0.040830,0.032139,0.023092,0.033885,0.039403,0.027820,0.025215,0.052047,0.038474,0.049140,...,0.009195,0.387193,0.613387,0.349942,0.500927,0.885366,0.436002,1.000000,0.775360,0.960826
